In [ ]:
from clipp2 import *
import pandas as pd
# File path
file_path = '/Users/yuding/Dropbox/GitHub/Multi_Region_CliPP/multi_clipp_simulation_data/simulation_data_cluster_3_region_1_read_depth_100_replica_2.tsv'

# Read the data into a DataFrame
df = pd.read_csv(file_path, sep='\t')

In [2]:
gamma = 0.05

In [3]:
rho = 0.8
omega = 1
n = df.shape[0]
m = 1
control_large = 5
precision = 0.01
max_iteration = 10000
sets = {i for i in range(n)}
combinations_2 = list(itertools.combinations(sets, 2))
pairs_mapping = {combination: index for index, combination in enumerate(combinations_2)}
pairs_mapping_inverse = {index: combination for index, combination in enumerate(combinations_2)}

# Get matrices
read_mat = get_read_mat(df)
total_read_mat = get_total_read_mat(df)
c_mat = get_c_mat(df)
bb_mat = get_b_mat(df)
tumor_cn_mat = get_tumor_cn_mat(df)
linearApprox = get_linear_approximation(c_mat)
# Initialize variables
#12/12/2024
phi_hat = (read_mat / (total_read_mat * c_mat))
scale_parameter = np.max([1, np.max(phi_hat)])
phi_hat = phi_hat / scale_parameter
phi_hat[phi_hat > sigmoid(control_large)] = sigmoid(control_large)
phi_hat[phi_hat < sigmoid(-control_large)] = sigmoid(-control_large)
p = inverse_sigmoid(phi_hat)
p[p > control_large] = control_large
p[p < -control_large] = -control_large
p = p.reshape([n * m])
v = np.zeros([len(combinations_2) * m])
for i in range(len(combinations_2)):
    pair = combinations_2[i]
    index_v = pairs_mapping[pair]
    start_v = index_v * m
    end_v = (index_v + 1) * m
    l1, l2 = pairs_mapping_inverse[index_v]
    a_mat = a_mat_generator(l1, l2, n, m)
    v[start_v: end_v] = matmul_by_torch(a_mat, p)
    
y = np.ones([len(combinations_2) * m])
omega = np.ones([len(combinations_2)])
k = 0

control_large = 5
wcut = np.array(linearApprox[0])
coef = np.array(linearApprox[1])
temp = 100


In [4]:
while k < max_iteration and precision < temp:
    No_mutation = n * m
    theta_hat = np.reshape(read_mat / total_read_mat, [No_mutation])

    theta = np.exp(p) * np.reshape(bb_mat, [No_mutation]) / (2 + np.exp(p) * np.reshape(tumor_cn_mat, [No_mutation]))

    A = np.sqrt(np.reshape(total_read_mat, [No_mutation])) * (
                (p <= wcut[:, 0]) * coef[:, 1] + (p >= wcut[:, 1]) * coef[:, 5] + (p > wcut[:, 0]) * (
                    p < wcut[:, 1]) * coef[:, 3] - theta_hat) / np.sqrt(theta * (1 - theta))
    B = np.sqrt(np.reshape(total_read_mat, [No_mutation])) * (
                (p <= wcut[:, 0]) * coef[:, 0] + (p >= wcut[:, 1]) * coef[:, 4] + (p > wcut[:, 0]) * (
                    p < wcut[:, 1]) * coef[:, 2]) / np.sqrt(theta * (1 - theta))

    linear = rho * get_v_mat(v, y, rho, combinations_2, pairs_mapping, n, m) - (B * A)

    Minv = 1 / (B ** 2 + No_mutation * rho)
    Minv_diag = np.diag(Minv)

    trace_g = -rho * np.sum(Minv)

    Minv_outer = np.outer(Minv,Minv)
    inverted = Minv_diag - (1 / (1 + trace_g) * (-rho) * Minv_outer)
    p_new    = matmul_by_torch(inverted, linear.T)
    p_new = p_new.reshape([No_mutation])
    p_new[p_new > control_large] = control_large
    p_new[p_new < -control_large] = -control_large
    p = p_new
    temp = 0
    for i in range(len(combinations_2)):
        pair = combinations_2[i]
        index_v = pairs_mapping[pair]
        start_v = index_v * m
        end_v = (index_v + 1) * m
        v[start_v: end_v] = update_v_SCAD(index_v, pairs_mapping_inverse, p, y, n, m, rho, omega[i], gamma)                
        y[start_v: end_v] = update_y(y[start_v: end_v], v[start_v: end_v], i, pairs_mapping_inverse, p, n, m, rho)
        l1, l2 = pairs_mapping_inverse[index_v]
        a_mat = a_mat_generator(l1, l2, n, m)
        temp = max(temp, np.linalg.norm(matmul_by_torch(a_mat, p) - v[start_v: end_v]))
    rho = 1.02 * rho
    k = k + 1
    print('\r', k, ',', temp, end="")

 31 , 0.009169550241429958

In [5]:
diff = np.zeros((n, n))
class_label = -np.ones(n)
class_label[0] = 0
group_size = [1]
labl = 1
least_mut = np.ceil(0.05 * n)
for i in range(1, n):
    for j in range(i):
        index_v = pairs_mapping[(j, i)]
        start_v = index_v * m
        end_v = (index_v + 1) * m
        diff[j, i] = np.linalg.norm(v[start_v: end_v]) if np.linalg.norm(v[start_v: end_v]) > 0.05 else 0
        diff[i, j] = diff[j, i]

for i in range(1, n):
    for j in range(i):
        if diff[j, i] == 0:
            class_label[i] = class_label[j]
            group_size[int(class_label[j])] += 1
            break
    if class_label[i] == -1:
        class_label[i] = labl
        labl += 1
        group_size.append(1)

        
# ----------------------------------------------------------
#  PART A: Refine small clusters by reassigning to closest
# ----------------------------------------------------------
tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])  
tmp_grp  = np.where(group_size == tmp_size)

refine = False
if tmp_size < least_mut:
    refine = True

while refine:
    refine = False
    # Find which SNVs belong to the smallest cluster (tmp_grp[0][0])
    smallest_cluster = tmp_grp[0][0]
    tmp_col = np.where(class_label == smallest_cluster)[0]
    
    for i in range(len(tmp_col)):
        mut_idx = tmp_col[i]

        # -- Gather distance from mut_idx to all other SNVs
        if mut_idx != 0 and mut_idx != (n - 1):
            tmp_diff = np.abs(
                np.concatenate((
                    diff[0:mut_idx, mut_idx].ravel(),
                    [100],  # placeholder for self
                    diff[mut_idx, (mut_idx+1):n].ravel()
                ))
            )
            # Increase distances to members of tmp_col so we don't pick same cluster
            tmp_diff[tmp_col] += 100

            # Put these adjusted distances back
            diff[0:mut_idx, mut_idx] = tmp_diff[0:mut_idx]
            diff[mut_idx, (mut_idx+1):n] = tmp_diff[(mut_idx+1):n]

        elif mut_idx == 0:
            # Edge case: first row
            tmp_diff = np.append(100, diff[0, 1:n])
            tmp_diff[tmp_col] += 100
            diff[0, 1:n] = tmp_diff[1:n]

        else: 
            # Edge case: last row (mut_idx == n-1)
            tmp_diff = np.append(diff[0:(n-1), n-1], 100)
            tmp_diff[tmp_col] += 100
            diff[0:(n-1), n-1] = tmp_diff[0:(n-1)]

        # -- Pick cluster of the SNV that has the smallest distance in tmp_diff
        ind = tmp_diff.argmin()

        # Decrement the old cluster size
        old_clust = int(class_label[mut_idx])
        group_size[old_clust] -= 1

        # Reassign to the cluster of 'ind'
        new_clust = int(class_label[ind])
        class_label[mut_idx] = new_clust
        group_size[new_clust] += 1

    # Check if there's still a too-small cluster
    tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
    tmp_grp  = np.where(group_size == tmp_size)
    refine   = (tmp_size < least_mut)
    
# ----------------------------------------------------------
#  PART B: Recompute cluster centroids
# ----------------------------------------------------------
labels = np.unique(class_label)

phi_out = np.zeros((len(labels), m))  # shape = (#clusters x m)

for i in range(len(labels)):
    cluster_id = labels[i]
    ind = np.where(class_label == cluster_id)[0]

    # Re-assign members to a contiguous labeling i (0..K-1)
    class_label[ind] = i

    # Weighted average (across multiple regions)
    numerator   = np.sum(phi_hat[ind, :] * total_read_mat[ind, :], axis=0)
    denominator = np.sum(total_read_mat[ind, :], axis=0)
    phi_out[i, :] = numerator / denominator


# ----------------------------------------------------------
#  PART C: Merge clusters with small difference in *scalar*
# ----------------------------------------------------------
if len(labels) > 1:
    # We'll define a single scalar "phi_scalar[i]" for each cluster i,
    # using the same coverage weighting but flattening *all* regions into 1 value.
    # This matches the single-region logic, i.e. we get "one phi" per cluster.

    def compute_cluster_scalar(phi_center):
        # For example, you might just take the mean of the m-dimensional centroid:
        # return np.mean(phi_center)
        
        # Or, to be consistent with single-region code's weighting logic,
        # we can multiply by sum over all m's coverage. But typically the centroid
        # is already coverage-weighted. So let's do a simple average:
        return np.mean(phi_center)  

    phi_scalar = np.array([compute_cluster_scalar(phi_out[i,:]) for i in range(len(labels))])

    # Sort those scalars
    sort_vals = np.sort(phi_scalar)
    phi_diff = sort_vals[1:] - sort_vals[:-1]
    min_val  = phi_diff.min()
    min_ind  = phi_diff.argmin()

    least_diff = 0.01
    while min_val < least_diff:
        # Identify the two clusters with the smallest difference
        smaller_val = sort_vals[min_ind]
        bigger_val  = sort_vals[min_ind+1]

        # Which cluster(s) have these centroid values?
        combine_ind    = np.where(phi_scalar == smaller_val)[0]
        combine_to_ind = np.where(phi_scalar == bigger_val)[0]

        # Reassign all members in 'combine_ind' to 'combine_to_ind' 
        # (just pick the first if there's more than one)
        c_from = combine_ind[0]
        c_to   = combine_to_ind[0]
        class_label[class_label == c_from] = c_to

        # Recompute the unique label set
        labels = np.unique(class_label)

        # Recompute phi_out for the new cluster set
        phi_out = np.zeros((len(labels), m))
        for idx, lbl in enumerate(labels):
            cluster_members = np.where(class_label == lbl)[0]
            # Re-map them to a contiguous cluster index 'idx'
            class_label[cluster_members] = idx

            numerator   = np.sum(phi_hat[cluster_members, :] * total_read_mat[cluster_members, :], axis=0)
            denominator = np.sum(total_read_mat[cluster_members, :], axis=0)
            phi_out[idx, :] = numerator / denominator

        if len(labels) == 1:
            break

        # Recompute the scalar representation for each cluster
        phi_scalar = np.array([compute_cluster_scalar(phi_out[i,:]) for i in range(len(labels))])

        # Sort and repeat
        sort_vals = np.sort(phi_scalar)
        phi_diff  = sort_vals[1:] - sort_vals[:-1]
        min_val   = phi_diff.min()
        min_ind   = phi_diff.argmin()
        
        
phi_res = np.zeros((n, m))
for k in range(len(labels)):
    # all SNVs that belong to cluster k
    idx_k = np.where(class_label == k)[0]
    phi_res[idx_k, :] = phi_out[k, :]

In [6]:
class_label

array([ 0.,  5.,  9.,  1.,  9.,  5.,  2.,  9.,  2.,  1.,  9.,  3.,  7.,
        5.,  3.,  3.,  7.,  3.,  2., 10.,  3.,  6.,  4.,  5.,  1.,  5.,
        8.,  4.,  4.,  2.,  0.,  9.,  6.,  0.,  3.,  1.,  3.,  1.,  1.,
        4.,  2.,  6.,  3.,  7., 10., 10.,  4.,  7.,  1.,  9.,  8.,  9.,
        7.,  1.,  9.,  3.,  8.,  9.,  2.,  5., 10.,  1.,  0.,  9.,  8.,
       10., 10.,  2.,  0.,  2.,  6.,  6.,  3.,  1.,  2.,  0.,  6.,  0.,
        9.,  8.,  7.,  0.,  8.,  1.,  0.,  3.,  8.,  4.,  3.,  9.,  2.,
        7.,  2.,  3.,  3.,  1.,  4.,  0.,  0.,  5.])

In [2]:
from clipp2 import *
snv_df = pd.read_csv('/Users/yuding/Downloads/sample1.snv.txt', sep='\t')
cn_df = pd.read_csv('/Users/yuding/Downloads/sample1.cna.txt', sep='\t')
purity = 0.9
sample_id="unknown_sample"
valid_snvs_threshold=0
diff_cutoff=0.1

In [3]:
preproc_res = preprocess_for_clipp2(
        snv_df, cn_df, purity, 
        sample_id=sample_id,
        valid_snvs_threshold=valid_snvs_threshold,
        diff_cutoff=diff_cutoff
    )

In [5]:
preproc_res['snv_df_final']

,chromosome_index,position,alt_count,ref_count
0,1,2,124,373
1,1,7,135,369
2,1,12,144,381
3,1,17,114,369
4,1,22,130,388
...,...,...,...,...
476,1,2412,235,565
477,1,2417,216,274
478,1,2422,237,514
479,1,2427,208,275


In [10]:
df_for_cliPP2, n, m = build_cliPP2_input(preproc_res)

In [11]:
from clipp2 import *
clipp2_result = CliPP2(
        df_for_cliPP2, rho, gamma, omega,
        n, m,
        max_iteration=max_iteration,
        precision=precision,
        control_large=control_large
    )

In [12]:
clipp2_result

{'phi': array([[0.47129228],
        [0.50144594],
        [0.50144594],
        [0.43466003],
        [0.47129228],
        [0.50144594],
        [0.57549579],
        [0.43466003],
        [0.47129228],
        [0.47129228],
        [0.47129228],
        [0.50144594],
        [0.57549579],
        [0.47129228],
        [0.47129228],
        [0.50144594],
        [0.47129228],
        [0.57549579],
        [0.47129228],
        [0.47129228],
        [0.57549579],
        [0.57549579],
        [0.47129228],
        [0.57549579],
        [0.50144594],
        [0.50144594],
        [0.47129228],
        [0.57549579],
        [0.50144594],
        [0.43466003],
        [0.57549579],
        [0.43466003],
        [0.47129228],
        [0.57549579],
        [0.50144594],
        [0.47129228],
        [0.50144594],
        [0.57549579],
        [0.57549579],
        [0.57549579],
        [0.57549579],
        [0.47129228],
        [0.43466003],
        [0.57549579],
        [0.43466003],
   